In [327]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [328]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("../CLEAN_DATE_AAPL_Official_DB.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head(5)


,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Open,Gain_Loss_High,Gain_Loss_Low,Gain_Loss_Close,Gain_Loss_Volume
0,0.0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0.128906,0.128348,0.128348,0,Gain,Gain,Gain,Gain,Gain
1,1.0,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss,Loss,Loss,Loss,Loss
2,2.0,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss,Loss,Loss,Loss,Loss
3,3.0,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0.00279,0.00279,0.00279,-19286400,Gain,Gain,Gain,Gain,Loss
4,4.0,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0.003349,0.003349,0.003349,-12992000,Gain,Gain,Gain,Gain,Loss


In [329]:
apple_df['Date'] = pd.to_datetime(apple_df['Date'])
apple_df['Date'] = apple_df['Date'].map(dt.datetime.toordinal)
apple_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,High_Change,Low_Change,Close_Change,Volume_Change,Gain_Loss_Open,Gain_Loss_High,Gain_Loss_Low,Gain_Loss_Close,Gain_Loss_Volume
0,0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0.128906,0.128348,0.128348,0,Gain,Gain,Gain,Gain,Gain
1,1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,-0.006696,-0.006696,-0.006696,-293148800,Loss,Loss,Loss,Loss,Loss
2,2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,-0.008929,-0.008929,-0.008929,-70156800,Loss,Loss,Loss,Loss,Loss
3,3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0.00279,0.00279,0.00279,-19286400,Gain,Gain,Gain,Gain,Loss
4,4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0.003349,0.003349,0.003349,-12992000,Gain,Gain,Gain,Gain,Loss


In [330]:
apple_adj_df = apple_df.drop(['High_Change','Low_Change','Close_Change','Volume_Change','Gain_Loss_High','Gain_Loss_Low','Gain_Loss_Close','Gain_Loss_Volume'], axis=1)
apple_adj_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0,0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,Gain
1,1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,Loss
2,2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,Loss
3,3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,Gain
4,4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,Gain


In [331]:
apple_adj_df.dtypes

Unnamed: 0        float64
Date                int64
Open              float64
High              float64
Low               float64
Close             float64
Adj Close         float64
Volume            float64
Open_Change        object
Gain_Loss_Open     object
dtype: object

In [332]:
apple_adj_df["Open_Change"] = pd.to_numeric(apple_adj_df.Open_Change,errors='coerce')

In [333]:
apple_adj_df.dtypes

Unnamed: 0        float64
Date                int64
Open              float64
High              float64
Low               float64
Close             float64
Adj Close         float64
Volume            float64
Open_Change       float64
Gain_Loss_Open     object
dtype: object

In [334]:
apple_adj_df.count()

Unnamed: 0        10226
Date              10227
Open              10226
High              10226
Low               10226
Close             10226
Adj Close         10226
Volume            10226
Open_Change       10226
Gain_Loss_Open    10227
dtype: int64

In [335]:
apple_adj_df.isnull().sum()

Unnamed: 0        1
Date              0
Open              1
High              1
Low               1
Close             1
Adj Close         1
Volume            1
Open_Change       1
Gain_Loss_Open    0
dtype: int64

In [336]:
apple_drop = apple_adj_df.dropna(how="any")

In [337]:
apple_drop.isna().sum()

Unnamed: 0        0
Date              0
Open              0
High              0
Low               0
Close             0
Adj Close         0
Volume            0
Open_Change       0
Gain_Loss_Open    0
dtype: int64

In [338]:
apple_adj_df.count()

Unnamed: 0        10226
Date              10227
Open              10226
High              10226
Low               10226
Close             10226
Adj Close         10226
Volume            10226
Open_Change       10226
Gain_Loss_Open    10227
dtype: int64

In [316]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
#from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_adj_df = apple_adj_df.copy()
apple_adj_df['Gain_Loss_Open'] = le.fit_transform(apple_adj_df['Gain_Loss_Open'])
apple_adj_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0,0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,0
1,1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,1
2,2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,1
3,3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,0
4,4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,0


In [317]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_adj_df['Gain_Loss_Open']), classes=[1,0])
apple_adj_df = apple_adj_df.copy()
apple_adj_df['Gain_Loss_Open'] = binarized_gain_loss
apple_adj_df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Open_Change,Gain_Loss_Open
0,0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0,0.128348,1
1,1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0,-0.006138,0
2,2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0,-0.008929,0
3,3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0,0.002232,1
4,4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0,0.003349,1


In [318]:
# Perform binary encoding of Season and Quarter columns
#apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
#apple_binary_encoded.head()

In [319]:
# Define features set
X = apple_adj_df.copy()
X = apple_adj_df.drop(["Gain_Loss_Open","Open_Change"], axis=1)
X.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume
0,0.0,723161,0.128348,0.128906,0.128348,0.128348,0.100751,469033600.0
1,1.0,723164,0.122210,0.122210,0.121652,0.121652,0.095495,175884800.0
2,2.0,723165,0.113281,0.113281,0.112723,0.112723,0.088485,105728000.0
3,3.0,723166,0.115513,0.116071,0.115513,0.115513,0.090676,86441600.0
4,4.0,723167,0.118862,0.119420,0.118862,0.118862,0.093304,73449600.0


In [320]:
# Define the target set.
y = apple_adj_df["Gain_Loss_Open"].ravel()
y[:5]

array([1, 0, 0, 1, 1])

In [321]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [322]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [323]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [324]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [16]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [17]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,5,0
Actual 1,1,5


In [18]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [19]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,5,0
Actual 1,1,5


Accuracy Score : 0.9090909090909091
Classification Report
              precision    recall  f1-score   support

        Gain       0.83      1.00      0.91         5
        Loss       1.00      0.83      0.91         6

    accuracy                           0.91        11
   macro avg       0.92      0.92      0.91        11
weighted avg       0.92      0.91      0.91        11



In [20]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.05941855, 0.08026179, 0.07167485, 0.09178666, 0.06466225,
       0.0775773 , 0.08421006, 0.44700769, 0.00986184, 0.00696427,
       0.0031209 , 0.00345385])

In [21]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.44700768733017987, 'Close_24hr_Change'),
 (0.09178665790839752, 'Low'),
 (0.08421006166280286, 'Volume'),
 (0.08026179139453313, 'Open'),
 (0.07757729573731513, 'Adj Close'),
 (0.0716748456832342, 'High'),
 (0.06466225382750514, 'Close'),
 (0.05941855254264199, 'Date'),
 (0.009861839123925369, 'Season_Spring'),
 (0.006964271431183198, 'Season_Winter'),
 (0.0034538482350982363, 'Quarter_Q2'),
 (0.003120895123183453, 'Quarter_Q1')]